In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
name_engine = create_engine('sqlite:///../../../dataset/us/us_ticker_list_with_name.db')
revenue_engine = create_engine('sqlite:///../../../dataset/us/us_ticker_revenue_mac.db')

In [4]:
# get the name list(arr)
name_df = pd.read_sql('TOTAL',con=name_engine)
name_list = name_df.Symbol.to_list()
# Clean the data
# 我们需要将没有这个数据的ticker从我们的list中去除，以提高下面程序运行的速度
for ticker in name_list:
    try:
        quarter_revenue = pd.read_sql('{}-Quarter'.format(ticker),con=revenue_engine)
    except:
        name_list.remove(ticker)

,Symbol,Company Name,Industry,Market Cap
14,AAPL,Apple,"Technology Hardware, Storage & Periph...","2,780.73B"


In [3]:
def select_by_revenue(targetdate = 0):
    pd.set_option('mode.chained_assignment', None)
    selected_df = pd.DataFrame(columns=['ticker','revenue_g1','revenue_g2','revenue_g3'])
    for ticker in name_list:
        try:
            quarter_revenue = pd.read_sql('{}-Quarter'.format(ticker),con=revenue_engine)
            if len(quarter_revenue) < targetdate + 7:
                print('{} data not enough'.format(ticker))
                continue
            targettime = quarter_revenue.iloc[targetdate].datetime
            quarter_revenue['revenue_growth'] = 0
            for i in range(len(quarter_revenue)-4):
                quarter_revenue.revenue_growth.iloc[i] = (quarter_revenue.revenue.iloc[i] - quarter_revenue.revenue.iloc[i+4]) / quarter_revenue.revenue.iloc[i+4]
            # fix the growth rate
            quarter_revenue['fixed_revenue_growth'] = 0
            for i in range(len(quarter_revenue)-1):
                    quarter_revenue.fixed_revenue_growth.iloc[i] = (quarter_revenue.revenue_growth.iloc[i] + quarter_revenue.revenue_growth.iloc[i+1]) / 2 
            # pick the ticker
            if quarter_revenue.iloc[targetdate].revenue_growth > quarter_revenue.iloc[targetdate+1].revenue_growth > quarter_revenue.iloc[targetdate+2].revenue_growth > 0.2 or quarter_revenue.iloc[targetdate].revenue_growth > 1:
                revenue_g1 = quarter_revenue.iloc[targetdate].revenue_growth
                revenue_g2 = quarter_revenue.iloc[targetdate+1].revenue_growth
                revenue_g3 = quarter_revenue.iloc[targetdate+2].revenue_growth
                temp = pd.DataFrame([[ticker,revenue_g1,revenue_g2,revenue_g3]],columns=['ticker','revenue_g1','revenue_g2','revenue_g3'])
                selected_df = pd.concat([selected_df,temp])
                print('{} selected'.format(ticker))
            elif quarter_revenue.iloc[targetdate].fixed_revenue_growth > quarter_revenue.iloc[targetdate+1].fixed_revenue_growth > quarter_revenue.iloc[targetdate+2].fixed_revenue_growth > 0.2 or quarter_revenue.iloc[targetdate].fixed_revenue_growth > 1:
                revenue_g1 = quarter_revenue.iloc[targetdate].fixed_revenue_growth
                revenue_g2 = quarter_revenue.iloc[targetdate+1].fixed_revenue_growth
                revenue_g3 = quarter_revenue.iloc[targetdate+2].fixed_revenue_growth
                temp = pd.DataFrame([[ticker,revenue_g1,revenue_g2,revenue_g3]],columns=['ticker','revenue_g1','revenue_g2','revenue_g3'])
                selected_df = pd.concat([selected_df,temp])
                print('{} selected'.format(ticker))
            else:
                # print(revenue_g1,revenue_g2,revenue_g3)
                print('{} not selected'.format(ticker))
        except:
            print('{} not exsit'.format(ticker))
    # 删除inf
    selected_df.replace([np.inf, -np.inf], np.nan,inplace=True)
    selected_df.dropna(axis=0, inplace=True)
    selected_df.index = range(len(selected_df))
    selected_df.to_csv('./results/selected_ticker_by_revenue_{}.csv'.format(targettime),index=None)          

In [ ]:
# run the code 
for i in range(8):
    select_by_revenue(i)

# Sample

In [15]:
pd.set_option('mode.chained_assignment', None)
targetdate = 7
ticker ='AAPL'
selected_df = pd.DataFrame(columns=['ticker','revenue_g1','revenue_g2','revenue_g3'])
quarter_revenue = pd.read_sql('{}-Quarter'.format(ticker),con=revenue_engine)
quarter_revenue['revenue_growth'] = 0
for i in range(len(quarter_revenue)-4):
    quarter_revenue.revenue_growth.iloc[i] = (quarter_revenue.revenue.iloc[i] - quarter_revenue.revenue.iloc[i+4]) / quarter_revenue.revenue.iloc[i+4]
# fix the growth rate
quarter_revenue['fixed_revenue_growth'] = 0
for i in range(len(quarter_revenue)-1):
        quarter_revenue.fixed_revenue_growth.iloc[i] = (quarter_revenue.revenue_growth.iloc[i] + quarter_revenue.revenue_growth.iloc[i+1]) / 2 
# pick the ticker
if quarter_revenue.iloc[targetdate].revenue_growth > quarter_revenue.iloc[targetdate+1].revenue_growth > quarter_revenue.iloc[targetdate+2].revenue_growth > 0.2 or quarter_revenue.iloc[targetdate].revenue_growth > 1:
    revenue_g1 = quarter_revenue.iloc[targetdate].revenue_growth
    revenue_g2 = quarter_revenue.iloc[targetdate+1].revenue_growth
    revenue_g3 = quarter_revenue.iloc[targetdate+2].revenue_growth
    temp = pd.DataFrame([[ticker,revenue_g1,revenue_g2,revenue_g3]],columns=['ticker','revenue_g1','revenue_g2','revenue_g3'])
    selected_df = pd.concat([selected_df,temp])
    print('{} selected'.format(ticker))
elif quarter_revenue.iloc[targetdate].fixed_revenue_growth > quarter_revenue.iloc[targetdate+1].fixed_revenue_growth > quarter_revenue.iloc[targetdate+2].fixed_revenue_growth > 0.2 or quarter_revenue.iloc[targetdate].fixed_revenue_growth > 1:
    revenue_g1 = quarter_revenue.iloc[targetdate].fixed_revenue_growth
    revenue_g2 = quarter_revenue.iloc[targetdate+1].fixed_revenue_growth
    revenue_g3 = quarter_revenue.iloc[targetdate+2].fixed_revenue_growth
    temp = pd.DataFrame([[ticker,revenue_g1,revenue_g2,revenue_g3]],columns=['ticker','revenue_g1','revenue_g2','revenue_g3'])
    selected_df = pd.concat([selected_df,temp])
    print('{} selected'.format(ticker))
else:
    # print(revenue_g1,revenue_g2,revenue_g3)
    print('{} not selected'.format(ticker))
selected_df

AAPL not selected


,ticker,revenue_g1,revenue_g2,revenue_g3


In [13]:
quarter_revenue = pd.read_sql('{}-Quarter'.format('AAL'),con=revenue_engine)
quarter_revenue['revenue_growth'] = 0.0
for i in range(len(quarter_revenue)-4):
    quarter_revenue.revenue_growth.iloc[i] = (quarter_revenue.revenue.iloc[i] - quarter_revenue.revenue.iloc[i+4]) / quarter_revenue.revenue.iloc[i+4]
quarter_revenue['fixed_revenue_growth'] = 0.0
for i in range(len(quarter_revenue)-1):
    quarter_revenue.fixed_revenue_growth.iloc[i] = (quarter_revenue.revenue_growth.iloc[i] + quarter_revenue.revenue_growth.iloc[i+1]) / 2 
quarter_revenue

,datetime,revenue,revenue_growth,fixed_revenue_growth
0,2022-06-30,13422.0,0.794865,1.007213
1,2022-03-31,8896.0,1.219561,1.280255
2,2021-12-31,9427.0,1.340949,1.583806
3,2021-09-30,8969.0,1.826662,2.718510
4,2021-06-30,7478.0,3.610358,1.540528
5,2021-03-31,4008.0,-0.529301,-0.586670
6,2020-12-31,4027.0,-0.644038,-0.688823
7,2020-09-30,3173.0,-0.733608,-0.798994
8,2020-06-30,1622.0,-0.864381,-0.529933
9,2020-03-31,8515.0,-0.195484,-0.080600


In [ ]:
pd.set_option('mode.chained_assignment', None)
selected_df = pd.DataFrame(columns=['ticker','revenue_g1','revenue_g2','revenue_g3'])
for ticker in name_list:
    try:
        # annual_revenue = pd.read_sql('{}-Annual'.format(ticker), con=revenue_engine)
        quarter_revenue = pd.read_sql('{}-Quarter'.format(ticker),con=revenue_engine)
        quarter_revenue['revenue_growth'] = 0.0
        for i in range(len(quarter_revenue)-4):
            quarter_revenue.revenue_growth.iloc[i] = (quarter_revenue.revenue.iloc[i] - quarter_revenue.revenue.iloc[i+4]) / quarter_revenue.revenue.iloc[i+4]
        quarter_revenue['fixed_revenue_growth'] = 0.0
        for i in range(len(quarter_revenue)-1):
            quarter_revenue.fixed_revenue_growth.iloc[i] = (quarter_revenue.revenue_growth.iloc[i] + quarter_revenue.revenue_growth.iloc[i+1]) / 2 
        # pick the ticker
        if quarter_revenue.iloc[0].revenue_growth > quarter_revenue.iloc[1].revenue_growth > quarter_revenue.iloc[2].revenue_growth > 0.2 or quarter_revenue.iloc[0].revenue_growth > 1:
            revenue_g1 = quarter_revenue.iloc[0].revenue_growth
            revenue_g2 = quarter_revenue.iloc[1].revenue_growth
            revenue_g3 = quarter_revenue.iloc[2].revenue_growth
            temp = pd.DataFrame([[ticker,revenue_g1,revenue_g2,revenue_g3]],columns=['ticker','revenue_g1','revenue_g2','revenue_g3'])
            selected_df = pd.concat([selected_df,temp])
            print('{} selected'.format(ticker))
        elif quarter_revenue.iloc[0].fixed_revenue_growth > quarter_revenue.iloc[1].fixed_revenue_growth > quarter_revenue.iloc[2].fixed_revenue_growth > 0.2 or quarter_revenue.iloc[0].fixed_revenue_growth > 1:
            revenue_g1 = quarter_revenue.iloc[0].fixed_revenue_growth
            revenue_g2 = quarter_revenue.iloc[1].fixed_revenue_growth
            revenue_g3 = quarter_revenue.iloc[2].fixed_revenue_growth
            temp = pd.DataFrame([[ticker,revenue_g1,revenue_g2,revenue_g3]],columns=['ticker','revenue_g1','revenue_g2','revenue_g3'])
            selected_df = pd.concat([selected_df,temp])
            print('{} selected'.format(ticker))
        else:
            print('{} not selected'.format(ticker))
    except:
        print('{} not exsit'.format(ticker))
selected_df.dropna(axis=0,inplace=True)
selected_df.to_csv('selected_ticker_by_revenue.csv',index=None)

In [15]:
selected_df

,ticker,revenue_g1,revenue_g2,revenue_g3


In [ ]:
quarter_revenue = pd.read_sql('AAL-Qarter',con=revenue_engine)
quarter_revenue